# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 1 2023 2:10PM,256416,19,ADV80011273,"AdvaGen Pharma, Ltd",Released
1,Feb 1 2023 2:08PM,256415,10,CTF0011624,"Citieffe, Inc.",Released
2,Feb 1 2023 2:08PM,256415,10,9103827,"Citieffe, Inc.",Released
3,Feb 1 2023 1:57PM,256414,12,20230202-NVL,SugarBear,Released
4,Feb 1 2023 1:42PM,256413,19,ADV80011274,"AdvaGen Pharma, Ltd",Released
5,Feb 1 2023 1:42PM,256413,19,ADV80011275,"AdvaGen Pharma, Ltd",Released
6,Feb 1 2023 1:42PM,256413,19,ADV80011233,"AdvaGen Pharma, Ltd",Released
7,Feb 1 2023 1:42PM,256413,19,ADV80011234,"AdvaGen Pharma, Ltd",Released
8,Feb 1 2023 1:42PM,256413,19,ADV80011235,"AdvaGen Pharma, Ltd",Released
9,Feb 1 2023 1:42PM,256413,19,ADV80011236,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,256412,Released,1
31,256413,Released,27
32,256414,Released,1
33,256415,Released,2
34,256416,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256412,NaN,3.0,1.0
256413,NaN,NaN,27.0
256414,NaN,NaN,1.0
256415,NaN,NaN,2.0
256416,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256352,6.0,13.0,1.0
256360,0.0,9.0,8.0
256362,2.0,13.0,0.0
256363,0.0,8.0,7.0
256370,0.0,1.0,37.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256352,6,13,1
256360,0,9,8
256362,2,13,0
256363,0,8,7
256370,0,1,37


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256352,6,13,1
1,256360,0,9,8
2,256362,2,13,0
3,256363,0,8,7
4,256370,0,1,37


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256352,6,13,1
1,256360,,9,8
2,256362,2,13,
3,256363,,8,7
4,256370,,1,37


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 1 2023 2:10PM,256416,19,"AdvaGen Pharma, Ltd"
1,Feb 1 2023 2:08PM,256415,10,"Citieffe, Inc."
3,Feb 1 2023 1:57PM,256414,12,SugarBear
4,Feb 1 2023 1:42PM,256413,19,"AdvaGen Pharma, Ltd"
31,Feb 1 2023 1:23PM,256412,19,"AdvaGen Pharma, Ltd"
35,Feb 1 2023 1:16PM,256411,10,Bio-PRF
36,Feb 1 2023 1:15PM,256410,10,Ivaoes Animal Health
38,Feb 1 2023 1:13PM,256409,10,Emerginnova
40,Feb 1 2023 1:11PM,256408,10,"Methapharm, Inc."
41,Feb 1 2023 1:08PM,256406,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 1 2023 2:10PM,256416,19,"AdvaGen Pharma, Ltd",,,1
1,Feb 1 2023 2:08PM,256415,10,"Citieffe, Inc.",,,2
2,Feb 1 2023 1:57PM,256414,12,SugarBear,,,1
3,Feb 1 2023 1:42PM,256413,19,"AdvaGen Pharma, Ltd",,,27
4,Feb 1 2023 1:23PM,256412,19,"AdvaGen Pharma, Ltd",,3,1
5,Feb 1 2023 1:16PM,256411,10,Bio-PRF,,,1
6,Feb 1 2023 1:15PM,256410,10,Ivaoes Animal Health,,,2
7,Feb 1 2023 1:13PM,256409,10,Emerginnova,,,2
8,Feb 1 2023 1:11PM,256408,10,"Methapharm, Inc.",,,1
9,Feb 1 2023 1:08PM,256406,10,ISDIN Corporation,,,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 2:10PM,256416,19,"AdvaGen Pharma, Ltd",1,,
1,Feb 1 2023 2:08PM,256415,10,"Citieffe, Inc.",2,,
2,Feb 1 2023 1:57PM,256414,12,SugarBear,1,,
3,Feb 1 2023 1:42PM,256413,19,"AdvaGen Pharma, Ltd",27,,
4,Feb 1 2023 1:23PM,256412,19,"AdvaGen Pharma, Ltd",1,3,
5,Feb 1 2023 1:16PM,256411,10,Bio-PRF,1,,
6,Feb 1 2023 1:15PM,256410,10,Ivaoes Animal Health,2,,
7,Feb 1 2023 1:13PM,256409,10,Emerginnova,2,,
8,Feb 1 2023 1:11PM,256408,10,"Methapharm, Inc.",1,,
9,Feb 1 2023 1:08PM,256406,10,ISDIN Corporation,6,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 2:10PM,256416,19,"AdvaGen Pharma, Ltd",1,,
1,Feb 1 2023 2:08PM,256415,10,"Citieffe, Inc.",2,,
2,Feb 1 2023 1:57PM,256414,12,SugarBear,1,,
3,Feb 1 2023 1:42PM,256413,19,"AdvaGen Pharma, Ltd",27,,
4,Feb 1 2023 1:23PM,256412,19,"AdvaGen Pharma, Ltd",1,3,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 2:10PM,256416,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
1,Feb 1 2023 2:08PM,256415,10,"Citieffe, Inc.",2.0,NaN,NaN
2,Feb 1 2023 1:57PM,256414,12,SugarBear,1.0,NaN,NaN
3,Feb 1 2023 1:42PM,256413,19,"AdvaGen Pharma, Ltd",27.0,NaN,NaN
4,Feb 1 2023 1:23PM,256412,19,"AdvaGen Pharma, Ltd",1.0,3.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 2:10PM,256416,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
1,Feb 1 2023 2:08PM,256415,10,"Citieffe, Inc.",2.0,0.0,0.0
2,Feb 1 2023 1:57PM,256414,12,SugarBear,1.0,0.0,0.0
3,Feb 1 2023 1:42PM,256413,19,"AdvaGen Pharma, Ltd",27.0,0.0,0.0
4,Feb 1 2023 1:23PM,256412,19,"AdvaGen Pharma, Ltd",1.0,3.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3333181,83.0,27.0,2.0
12,512815,1.0,1.0,0.0
15,512723,15.0,17.0,0.0
19,1794771,39.0,16.0,6.0
21,512783,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3333181,83.0,27.0,2.0
1,12,512815,1.0,1.0,0.0
2,15,512723,15.0,17.0,0.0
3,19,1794771,39.0,16.0,6.0
4,21,512783,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,83.0,27.0,2.0
1,12,1.0,1.0,0.0
2,15,15.0,17.0,0.0
3,19,39.0,16.0,6.0
4,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,83.0
1,12,Released,1.0
2,15,Released,15.0
3,19,Released,39.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,21
Status,,,,,
Completed,2.0,0.0,0.0,6.0,0.0
Executing,27.0,1.0,17.0,16.0,0.0
Released,83.0,1.0,15.0,39.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,21
0,Completed,2.0,0.0,0.0,6.0,0.0
1,Executing,27.0,1.0,17.0,16.0,0.0
2,Released,83.0,1.0,15.0,39.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,21
0,Completed,2.0,0.0,0.0,6.0,0.0
1,Executing,27.0,1.0,17.0,16.0,0.0
2,Released,83.0,1.0,15.0,39.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()